In [ ]:
import codecs 
import requests 
from bs4 import BeautifulSoup as bs 
from slugify import slugify 
import pickle 
import time 
import os 

In [ ]:
SearchTemplate = 'https://pubmed.ncbi.nlm.nih.gov/?term=Sars-Cov-2%20or%20COVID-19%20or%20nCOV-2019&size=100&page={}'

In [ ]:
# Download and cache a webpage if it hasn't been downloaded and cached before
# NOTE: Requires the existence of a folder named 'searchBioPages'!
def cacheQuery(query, forceUncache=False):
    queryFile = 'searchBioPages/' + slugify(query)
    if ((not forceUncache) and os.path.isfile(queryFile)):
        data = pickle.load(open(queryFile, 'rb'))
    else:
        time.sleep(1) # Waiting 1 second is the minimum level of "politeness"
        r = requests.get(query)
        if (r.status_code == 200):
            data = r.text
            pickle.dump(data, open(queryFile, 'wb'))  
        else:
            data = None
    return data

In [ ]:
# Parse each search page for article listings
def processSearchPage(page, query):

    articleData = [] # The article listings for this page, indexed by DOI

    html = bs(page, "lxml") # Initialize BeautifulSoup parser with lxml parsing module
    
  
    article_lists = html.find('div', attrs={'class':'search-results-chunk results-chunk'})
    article_markers = str(article_lists)[64:963]
    articles = article_markers.replace(',','')
    articles = articles.split()
    articles = str(articles)[2:802]
    i = 0
    while i <len(str(articles)):
        articleData.append(articles[i:i+8])
        i = i+8
    return articleData 

In [ ]:
# Call the functions below to open a search page and copy the urls for the articles therein
def getUrlsInRange(page_num):

    global Create_file
    
    queryString = SearchTemplate.format(page_num)
    page = cacheQuery(queryString)

    if (page is not None):
        ArticlePointers = processSearchPage(page, queryString)
    
    # Write all urls into a fileall the DOI info to a file
        i = 0
        while i < len(ArticlePointers):
            Create_file.write(ArticlePointers[i] + '\n')
            i = i+1

In [ ]:
# Get the Urls for the articles in the search

Create_file = codecs.open("links_comprehensive_pubmed_covid-19.txt", 'w', 'utf-8')
FirstPage = 1
LastPage = 196

# Set the start and end of the search pages
d = FirstPage
delta = 1

# Step through the full page range, incrementing the start and end

while d <= LastPage:
    searchPage = d
    getUrlsInRange(searchPage)
    d += delta

Create_file.close()

In [ ]:
#Prep links for extraction of data
urls =[]
with open("links_comprehensive_pubmed_covid-19.txt","r") as links:
    for line in links:
        urls.append(str(line)[:8])

In [ ]:
#Extract abstracts from links

abstractData = {}
abstract = codecs.open("abstracts_compre_pubmed_covid-19.txt","w",encoding = "utf-8") 
item = 0
while item < len(urls):
    page_file = requests.get('https://pubmed.ncbi.nlm.nih.gov/'+str(urls[item]))
    if (page_file.status_code == 200):
        page_file = page_file.text
        data = bs(page_file,'lxml')
        abstract_file = data.find('div', attrs={'class':'abstract-content selected'})
        if abstract_file is not None:
            if abstract_file.p is not None:
                if abstract_file.p.text is not None:
                    abstractData[urls[item]] = abstract_file.p.text.strip()
                    abstract.write(str(abstract_file.p.text.strip()) +"\n")  
    item = item+1
abstract.close()